In [1]:
import pandas as pd
import numpy as np
from surprise import SlopeOne
from surprise import accuracy
from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split
from surprise.model_selection import KFold

from read_and_split_data import split_data, filter_animes_without_grade

In [2]:
def get_dataset():
    anime = pd.read_parquet("../datasets/anime.parquet")
    anime = anime[["anime_id", "type"]]
    users = pd.read_parquet("../datasets/users.parquet")
    base_df = users.merge(anime, on="anime_id", how="left")
    return base_df

base_df = get_dataset()
data = split_data(base_df)

In [3]:
kf = KFold(n_splits=5)

In [4]:
def predict_and_error(data, algo, path):
    fold = 0
    predictions_list = []
    for trainset, testset in kf.split(data):
        algo.fit(trainset)
        predictions = algo.test(testset)
        predictions_list.append(predictions)
        # Mean Squared Error
        accuracy.rmse(predictions, verbose=True)
        # Mean Absolute Erro
        accuracy.mse(predictions, verbose=True)
        # Mean Absolute Erro
        accuracy.mae(predictions, verbose=True)
        # Fraction of Concordant Pairs
        accuracy.fcp(predictions, verbose=True)
        base_df = pd.DataFrame(columns=["user", "anime", "actual", "est", "details"], data=predictions)
        base_df.to_csv(f"{path}_{fold}.csv")
        base_df.to_parquet(f"{path}_{fold}.parquet")
        fold += 1
    return predictions

In [5]:
algo_normal_predictor = SlopeOne()

In [6]:
predictions = predict_and_error(data, algo_normal_predictor, path="../predictions/slope_one/anime_type_tv/no_filter/slope_one")

RMSE: 2.1772
MSE: 4.7401
MAE:  1.4950
FCP:  0.6898
RMSE: 2.1731
MSE: 4.7225
MAE:  1.4927
FCP:  0.6889
RMSE: 2.1706
MSE: 4.7117
MAE:  1.4914
FCP:  0.6886
RMSE: 2.1714
MSE: 4.7150
MAE:  1.4915
FCP:  0.6888
RMSE: 2.1724
MSE: 4.7192
MAE:  1.4932
FCP:  0.6882


In [7]:
base_df_without_negative = filter_animes_without_grade(base_df)

In [8]:
predictions_without_negative = predict_and_error(data, algo_normal_predictor, path="../predictions/slope_one/anime_type_tv/with_filter_remove_negative/slope_one")

RMSE: 2.1766
MSE: 4.7377
MAE:  1.4939
FCP:  0.6884
RMSE: 2.1705
MSE: 4.7110
MAE:  1.4910
FCP:  0.6891
RMSE: 2.1702
MSE: 4.7098
MAE:  1.4930
FCP:  0.6900
RMSE: 2.1726
MSE: 4.7201
MAE:  1.4921
FCP:  0.6891
RMSE: 2.1742
MSE: 4.7273
MAE:  1.4939
FCP:  0.6881
